<a href="https://colab.research.google.com/github/abeni-hub/Machine_Learning-Project-Collections/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
!apt install wget

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
dataset.info()

In [ ]:
# Replacing string values to numbers
dataset['sex'] = dataset['sex'].replace(['female', 'male'], [0, 1])
dataset['smoker'] = dataset['smoker'].replace(['no', 'yes'], [0, 1])
dataset['region'] = dataset['region'].apply({'southwest':1,'southeast':2 ,'northwest':3,'northeast': 4}.get)

In [ ]:
# Encoding categorial data
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(),[1,4,5])], remainder='passthrough')
dataset = np.array(ct.fit_transform(dataset))

In [ ]:
# splitting the dataset into the Training Set and Test Set
from sklearn.model_selection import train_test_split
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=0)

In [ ]:
# TO dataframe inorder to pop
train_dataset = pd.DataFrame(train_dataset)
test_dataset = pd.DataFrame(test_dataset)

In [ ]:
# Pop the 'expenses' column
train_labels = train_dataset.pop(train_dataset.columns[-1])
test_labels = test_dataset.pop(test_dataset.columns[-1])

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train_dataset = sc.fit_transform(train_dataset)
test_dataset = sc.transform(test_dataset)

In [ ]:
# BUILDING THE MODEL

# Initializing the Model
model = tf.keras.models.Sequential()

# Adding the Input Layer and the first hidden layer
model.add(tf.keras.layers.Dense(units=6, activation='relu', input_shape=[train_dataset.shape[1]]))
model.add(tf.keras.layers.Dense(units=6, activation='relu'))

# Adding the Output Layer
model.add(tf.keras.layers.Dense(units=1 , activation = 'linear'))

# Compiling the Model
from keras.optimizers import SGD
opt_einsum = SGD(learning_rate=0.01)
model.compile(optimizer=opt_einsum, loss='mean_squared_error', metrics=['mae','mse'])

model.fit(train_dataset, train_labels,
          batch_size = 32,
          epochs = 100,
          validation_data = (test_dataset, test_labels),
          verbose = 2)


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
